In [0]:
!pip install keras-bert
!pip install keras-xlnet
!pip install tokenization
!pip install bert
!pip install keras-lr-finder
!pip install bert-tensorflow
!pip install numpy==1.16.2

In [1]:
%tensorflow_version 1.14
import os
# os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import re
import string
import matplotlib.pyplot as plt
%matplotlib inline
import copy
import cv2
from os import listdir
from sklearn.metrics import accuracy_score
from PIL import Image

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import backend as K

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) #command to run codeon multiple gpu

from keras_lr_finder import LRFinder
from pprint import pprint
tf.logging.set_verbosity(tf.logging.ERROR)
from random import choice
import gc
from bert import tokenization
from keras_xlnet.backend import keras
from keras_bert.layers import Extract
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint, ATTENTION_TYPE_BI

from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/data")

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
np.random.seed(42)
tf.set_random_seed(42)

In [0]:
data_image = np.load("data_image_1.npy")
data_text = np.load("data_text_1.npy")
data_label = np.load("data_label_1.npy")

In [0]:
length = int(len(data_image) * 0.8)
train_images = data_image[:15360]
train_text = data_text[:15360]
trainY = data_label[:15360]
test_images = data_image[15362:]
test_text = data_text[15362:]
testY = data_label[15362:]

In [0]:
train_imagesX = np.load('train_imagesX.npy')
test_imagesX = np.load('test_imagesX.npy')

In [6]:
len(train_images),len(test_images), len(train_text), len(test_text), len(trainY), len(testY), len(train_imagesX)

(15360, 3840, 15360, 3840, 15360, 3840, 15361)

In [0]:
EPOCH = 5
BATCH_SIZE = 16
SEQ_LEN = 196

In [8]:
checkpoint_path = 'chinese_xlnet_base_L-12_H-768_A-12'

tokenizer = Tokenizer(os.path.join(checkpoint_path, 'spiece.model'))
xlnet = load_trained_model_from_checkpoint(
    config_path=os.path.join(checkpoint_path, 'xlnet_config.json'),
    checkpoint_path=os.path.join(checkpoint_path, 'xlnet_model.ckpt'),
    batch_size=BATCH_SIZE,
    memory_len=0,
    target_len=SEQ_LEN,
    in_train_phase=False,
    attention_type=ATTENTION_TYPE_BI,
)
xlnet.trainable = False
text_output = xlnet.output
extract = Extract(index=-1, name='Extract')(text_output)
text_repr = keras.layers.Dense(32, activation='relu')(extract)

output = keras.layers.Dense(1, activation='sigmoid')(text_repr)

model = keras.models.Model(inputs=[xlnet.input[0], xlnet.input[1], xlnet.input[2]], outputs=output)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 196)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 196, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 196)          0           Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Token-Masked (RestoreMask (None, 196, 768)     0           Embed-Token[0][0]                
                                                                 Masking[0][0]              

In [0]:
def generate_sequence(text, Y):
    tokens, classes = [], []
    
    for i in text:
        encoded = tokenizer.encode(i)[:SEQ_LEN - 1]
        encoded = [tokenizer.SYM_PAD] * (SEQ_LEN - 1 - len(encoded)) + encoded + [tokenizer.SYM_CLS]
        tokens.append(encoded)

    tokens, classes = np.array(tokens), Y
    segments = np.zeros_like(tokens)
    segments[:, -1] = 1
    lengths = np.zeros_like(tokens[:, :1])

    return (np.array(tokens), np.array(segments), np.array(lengths), np.array(classes))


train_tokens, train_segments, train_lengths, trainy = generate_sequence(train_text, trainY)

test_tokens, test_segments, test_lengths, testy = generate_sequence(test_text, testY)

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00005),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

### 定义callback函数，只保留val_sparse_categorical_accuracy 得分最高的模型

from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("MMFFND-XLNet-Finetune.h5", monitor='accuracy', verbose=1, save_best_only=True,
                            mode='max')

model.fit(
    x=[train_tokens, train_segments, train_lengths], 
    y=trainy,
    batch_size=BATCH_SIZE,
    epochs=5,
    verbose=1,
    shuffle=False,
    validation_split=0.3,
    callbacks=[checkpoint]
)

Train on 10752 samples, validate on 4608 samples
Epoch 1/5
10752/10752 [==============================] - 400s 37ms/step - loss: 0.1683 - accuracy: 0.9304 - val_loss: 0.1245 - val_accuracy: 0.9499

Epoch 00001: accuracy improved from -inf to 0.93043, saving model to MMFFND-XLNet-Finetune.h5
Epoch 2/5
10752/10752 [==============================] - 385s 36ms/step - loss: 0.0416 - accuracy: 0.9855 - val_loss: 0.0718 - val_accuracy: 0.9789

Epoch 00002: accuracy improved from 0.93043 to 0.98549, saving model to MMFFND-XLNet-Finetune.h5
Epoch 3/5
10752/10752 [==============================] - 385s 36ms/step - loss: 0.0240 - accuracy: 0.9920 - val_loss: 0.0912 - val_accuracy: 0.9698

Epoch 00003: accuracy improved from 0.98549 to 0.99200, saving model to MMFFND-XLNet-Finetune.h5
Epoch 4/5
10752/10752 [==============================] - 385s 36ms/step - loss: 0.0149 - accuracy: 0.9942 - val_loss: 0.1288 - val_accuracy: 0.9696

Epoch 00004: accuracy improved from 0.99200 to 0.99423, saving mode

In [0]:
test_predict = model.predict([test_tokens, test_segments, test_lengths],batch_size=16)
test_predict = [1 if i>=0.5 else 0 for i in test_predict]

In [12]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
print(f'Accuracy == {accuracy_score(testY,test_predict)}')
print(f'F1 == {f1_score(testY,test_predict,average=None)}')
print(f'Precision == {precision_score(testY,test_predict,average=None)}')
print(f'Recall == {recall_score(testY,test_predict,average=None)}')

Accuracy == 0.9734375
F1 == [0.97371134 0.97315789]
Precision == [0.96377551 0.98351064]
Recall == [0.98385417 0.96302083]


In [0]:
fault = []
for i in range(len(test_predict)):
    if test_predict[i] != testY[i]:
        fault.append(i)

In [14]:
len(fault)

96

In [15]:
for i in fault:
    print(test_text[i] + "  predict: " + str(test_predict[i]) + ", truth: " + str(int(testY[i])) + "\n")

#警方通报空姐遇害案#朋友圈转被抓到了？不知道真的假的，真希望快抓到！！！  predict: 0, truth: 1

找人脉，寻商机，人脉通企业营销利器！分享钢材|钢铁行业王友林的动态：广西南宁市303医院：昨天凌晨二点二十三分，三名男女生感染h7n9病毒死亡，最大的32岁，...—来自&gt;&gt;分享自@脉否-人脉通  predict: 0, truth: 1

哼哼~给你们原图。  predict: 0, truth: 1

广电总局2015年管令：1.4月起实施限外令，海外电视剧将延迟半年播出2.演员名人不得主持养生节目3.2015年起一部电视剧将最多在两家卫视联播4.9月到10月底间，必须播爱国主义题材和反法西斯题材。5.严打婚外恋、一夜情内容6.青春校园片可出现早恋但不许成功7.建国后动物不许修炼成精。  predict: 0, truth: 1

网传四川成都至宜宾高速，500车连撞，堪称世界第一大车祸。祈祷#四川成都至宜宾高速,500车连撞#  predict: 0, truth: 1

分享网易视频《差点就抓住！3岁男童从商场扶梯处坠亡，父母就在几米外》来自@网易新闻客户端#网易新闻#  predict: 1, truth: 0

山竹台风天，我第一次上传说了。奇迹牧。  predict: 1, truth: 0

南京新街口莱迪广场篷布被雪压塌了。  predict: 1, truth: 0

#南京唐僧肉#你们看看这个合法不  predict: 1, truth: 0

网传12306密码疑似被脱库，谨慎的你还不去修改下密码？  predict: 0, truth: 1

#台风玛莉亚#“玛利亚”登陆连江爱不离2的...  predict: 1, truth: 0

重要通知！截止到09.27早上10:00孩子仍未找到请朋友圈的各位继续帮忙寻找孩子勿传谣，以免对找寻工作造成影响！@汕头快乐123@汕头民声热线@汕头同城@汕头电台英子2汕头·岐山街区  predict: 1, truth: 0

#警方通报空姐遇害案#法网恢恢，害人之心不可有！  predict: 0, truth: 1

有人想等女友跑完马拉松后向她求婚，不料她却惨遭丧生。好好珍惜身边的人，你永远不知道明天和意外哪个会先来。  predict: 0, truth: 

In [0]:
model.save("/content/gdrive/My Drive/Colab Notebooks/data/MMFFND-XLNet.h5")

In [0]:
bert = np.load("/content/gdrive/My Drive/Colab Notebooks/data/bert_predict.npy")

In [0]:
bert

array([[1.0000000e+00],
       [3.2156706e-05],
       [1.0000000e+00],
       ...,
       [7.3015690e-06],
       [5.3158402e-04],
       [1.0000000e+00]], dtype=float32)

In [0]:
xlnet = np.array(test_predict)

In [0]:
out = (bert[1:] + xlnet) / 2
out = [1 if i>=0.5 else 0 for i in out]

In [0]:
print(f'Accuracy == {accuracy_score(testY,out)}')
print(f'F1 == {f1_score(testY,out,average=None)}')
print(f'Precision == {precision_score(testY,out,average=None)}')
print(f'Recall == {recall_score(testY,out,average=None)}')

NameError: ignored